# Parallelism

More efficient algorithms are more powerful algorithms in that they can solve larger problems than can less efficient versions of them. Many of our world's most important problems require tremendous computational power, more than any individual CPU could provide in any reasonable timeframe. 

A parallel algorithm is one in which independent parts of the problem can be solved concurrently and their solutions combined into a final solution. Parallel algorithms spread their work over multiple cores, allowing computational resources to be used more effectively and larger problems to be solved in the same timeframe.

In this section, we will introduce parallel algorithms and fundamental concepts for designing and analyzing them. 


